This notebook is associated with the paper "The relative class number one problem for function fields, I" by K.S. Kedlaya. It runs in SageMath (tested using version 9.5) and depends on Magma (tested using version
2.26-9); it also requires the pandas and openpyxl libraries (`sage --pip install openpyxl/pandas`).

In this notebook, we compute cyclic purely geometric extensions with $q=2$ using the upper bound on Weil polynomials. This depends on the exhaustion over Weil polynomials performed in the notebook "Weil polynomial bound for q=2" in this repository, the tabulation of curves of genus $\leq 4$ in LMFDB (including data by Xarles in genus 4), and the data file "genus5byweilpoly.txt" created by the notebook "Genus 5 curves.ipynb" based on the data of Dragutinović in genus 5. 

Allow 1 hour for completion when `long == False`. When `long == True`, we run a consistency check where we try all possible base curves instead of using the calculated possibilities.

In [1]:
load("../Shared/weil_poly_utils.sage")
load("../Shared/cyclic_covers.sage")
load("../Shared/function_fields.sage")

In [2]:
long = False

# Setup

Read data about the exhaustion from an Excel spreadsheet. Note that we only use the data about base curves $C$, not covering curves $C'$ (except at the end to match up the results).

In [3]:
import pandas, itertools, re, pickle
from collections import defaultdict

In [4]:
df = pandas.read_excel('../Shared/polys.xlsx', index_col=int(0))

In [5]:
if "Unnamed: 0" in df:
    del df["Unnamed: 0"]
if "Cyclic" in df:
    del df["Cyclic"]
print(list(df))

['d', 'g', "g'", 'Label of J(C)', '#J(C)(F_2)', '#J(C)(F_4)', 'Counts of C', "Counts of C'"]


In [6]:
candidates = {}
for i in range(len(df)):
    r = df.iloc[i]
    d = r["d"]
    g = r["g"]
    g1 = r["g'"]
    if (d,g,g1) not in candidates:
        candidates[d,g,g1] = []
    s = r["Label of J(C)"]
    if s not in candidates[d,g,g1]:
        candidates[d,g,g1].append(s)

# Construction of extensions

Construct extensions with relative class number 1.

If `long == True`, ignore the Weil polynomial bound and instead try *all* base curves of the specified genus.

In [7]:
def match_weil_poly_general(s, d, delta=0, verbose=False):
    l = s.split(".")
    g = Integer(l[0])
    g1 = g + (d-1)*(g-1) + delta
    ans = []
    for F in function_fields_by_weil_poly(s):
        ans.append((s, match_weil_poly(F, d, delta=delta, verbose=verbose)))
    if ans:
        if (d, g, g1) not in covers_by_poly:
            covers_by_poly[d, g, g1] = {}
        if s not in covers_by_poly[d, g, g1]:
            covers_by_poly[d, g, g1][s] = {}
        for (_, l) in ans:
            for (F, F1) in l:
                if F not in covers_by_poly[d, g, g1][s]:
                    covers_by_poly[d, g, g1][s][F] = []
                if F1 not in covers_by_poly[d, g, g1][s][F]:
                    covers_by_poly[d, g, g1][s][F].append(F1)
    return ans

In [8]:
covers_by_poly = {}

In [9]:
P.<T> = QQ[]
for (d, g, g1) in candidates:
    print(d, g, g1)
    delta = g1-g-(d-1)*(g-1)
    covers_by_poly[d, g, g1] = {}
    for s in curves_by_poly[g] if long else candidates[d,g,g1]:
        u = weil_poly_from_label(P, s)
        if delta or (u(1) % d == 0):
            match_weil_poly_general(s, d, delta=g1-g-(d-1)*(g-1))

2 2 3
2 2 4
2 2 5
2 3 5
2 3 6
2 4 7
2 4 8
2 5 9
2 6 11
2 7 13
3 2 4
3 2 6
3 3 7
3 4 10
4 2 5
4 2 6
4 3 9
5 2 6
6 2 7
7 2 8


Verify that no base field is repeated.

In [10]:
for (d, g, g1) in covers_by_poly:
    print(d,g,g1)
    for s in covers_by_poly[d,g,g1]:
        if covers_by_poly[d,g,g1][s]:
            l1 = list(covers_by_poly[d,g,g1][s].keys())
            l2 = isomorphism_class_reps(l1)
            assert len(l1) == len(l2)

2 2 3
2 2 4
2 2 5
2 3 5
2 3 6
2 4 7
2 4 8
2 5 9
2 6 11
2 7 13
3 2 4
3 2 6
3 3 7
3 4 10
4 2 5
4 2 6
4 3 9
5 2 6
6 2 7
7 2 8


Verify that each base curve gives rise to a unique cover of the specified degree and genus.

In [11]:
for (d, g, g1) in covers_by_poly:
    print(d,g,g1)
    for s in covers_by_poly[d,g,g1]:
        for F, l1 in covers_by_poly[d,g,g1][s].items():
            if len(l1) > 1:
                assert len(l1) == 2
                F1 = l1[0].RationalExtensionRepresentation()
                F2 = l1[1].RationalExtensionRepresentation()
                assert F1.IsIsomorphic(F2)

2 2 3
2 2 4
2 2 5
2 3 5
2 3 6
2 4 7
2 4 8
2 5 9
2 6 11
2 7 13
3 2 4
3 2 6
3 3 7
3 4 10
4 2 5
4 2 6
4 3 9
5 2 6
6 2 7
7 2 8


Count cyclic covers arising from this calculation.

In [12]:
sum(1 for (d,g,g1) in covers_by_poly for s in covers_by_poly[d,g,g1] for l1 in covers_by_poly[d,g,g1][s].values() if l1)

64

# Populating the spreadsheet

Convert the covers into numerical data in preparation for writing out to the spreadsheet.

In [13]:
l = []
for (d, g, g1) in candidates:
    print(d, g, g1)
    if (d, g, g1) in covers_by_poly:
        for s in covers_by_poly[d, g, g1]:
            for F in covers_by_poly[d, g, g1][s]:
                for F1 in covers_by_poly[d, g, g1][s][F]:
                    u1 = T.parent()(magma_poly_list(F.ZetaFunction().Numerator())).reverse()
                    v1 = T.parent()(magma_poly_list(F1.ZetaFunction().Numerator())).reverse()
                    l.append((d, g, g1, point_count_from_weil_poly(u1, 13),
                              point_count_from_weil_poly(v1, 13)))

2 2 3


KeyError: Algebraic function field defined over Univariate rational function field over GF(2) by
y^2 + x*y + x^5 + x^3 + x

In [ ]:
covers_by_poly[2,2,3]['2.2.b_c']

Write the results back to the spreadsheet. In the process, we check that every covering we found gives a pair of Weil polynomials from our original list.

In [ ]:
df['Cyclic'] = 'No'
used_pols = []
for i in range(len(df)):
    r = df.iloc[i]
    d = r["d"]
    g = r["g"]
    g1 = r["g'"]
    ct1 = eval(r["Counts of C"])
    ct2 = eval(r["Counts of C'"])
    if (d, g, g1, ct1, ct2) in l:
        df.loc[i, "Cyclic"] = "Yes"
        used_pols.append((d, g, g1, ct1, ct2))
assert all(t in used_pols for t in l)

In [ ]:
df.to_excel('../Shared/polys.xlsx', sheet_name='Weil polynomials', merge_cells=True, freeze_panes=(int(1),int(1)))

# Tables

Generate LaTeX-formatted tables of the data for $d=2$ and $d>2$.

In [ ]:
R0.<x> = GF(2)[]
R1.<y> = R0[]
print(r'\begin{tabular}{c|c|c|c|c}')
print(r"$d$ & $g_F$ & $g_{F'}$ & $J(C)$ & $F$ \\")
for (d, g, g1) in candidates:
    if d > 2:
        if any(covers_by_poly[d, g, g1][s] for s in covers_by_poly[d, g, g1]):
            print(r'\hline')
            for s in covers_by_poly[d, g, g1]:
                if covers_by_poly[d,g,g1][s]:
                    for F in covers_by_poly[d, g, g1][s]:
                        F1 = F.RationalExtensionRepresentation()
                        try:
                            u = F1.DefiningPolynomial().sage()
                        except SyntaxError:
                            u = magma_poly_list(F.DefiningPolynomial())
                            u = [R0(magma_poly_list(i.Numerator())) / R0(magma_poly_list(i.Denominator())) for i in u]
                            u = R1(u)
                        print(r'${}$ & ${}$ & ${}$ & $\avlink{{{}}}$ & ${}$ \\'.format(d, g, g1,
                                                        re.sub('_', r'\_', s),
                                                        latex(u)))
print(r'\end{tabular}')

In [ ]:
R0.<x> = GF(2)[]
R1.<y> = R0[]
print(r'\begin{tabular}{c|c|c|p{7.5cm}}')
print(r"$g_F$ & $g_{F'}$ & $J(C)$ & $F$ \\")
for (d, g, g1) in candidates:
    if d == 2:
        if covers_by_poly[d, g, g1]:
            print(r'\hline')
            if g == 6:
                print(r'\hline')
            for s in covers_by_poly[d, g, g1]:
                if covers_by_poly[d,g,g1][s]:
                    for F in covers_by_poly[d, g, g1][s]:
                        F1 = F.RationalExtensionRepresentation()
                        try:
                            u = F1.DefiningPolynomial().sage()
                        except SyntaxError:
                            u = magma_poly_list(F1.DefiningPolynomial())
                            u = [R0(magma_poly_list(i.Numerator())) / R0(magma_poly_list(i.Denominator())) for i in u]
                            u = R1(u)
                        if g < 7: 
                            s1 = r"\avlink{{{}}}".format(s)
                        else:
                            s1 = str(tuple(Integer(F.NumberOfPlacesOfDegreeOneECF(i)) for i in range(1, g+1)))
                        print(r'${}$ & ${}$ & ${}$ & ${}$ \\'.format(g, g1,
                                                        re.sub('_', r'\_', s1),
                                                        latex(u)))
print(r'\end{tabular}')